In [1]:
import os
os.chdir("../")
print(os.getcwd())

/rds/general/user/pg221/home/PhD_projects2/dreamer_flows/dreamer


In [2]:
import warnings
import dreamerv3
from dreamerv3 import embodied
warnings.filterwarnings('ignore', '.*truncated to dtype int32.*')

print("Reading config", flush =True)

# See configs.yaml for all options.
config = embodied.Config(dreamerv3.configs['defaults'])
config = config.update(dreamerv3.configs['small'])
config = config.update({
  'logdir': '~/PhD_projects2/dreamer_flows/dreamer/logdir/notebook_test_cyl_ks_9',
  'run.train_ratio': 32,
  'run.log_every': 30,  # seconds
  'batch_size': 16,
  'batch_length': 8,
  'jax.prealloc': False,
    
  'encoder.mlp_keys': 'vector',
#   'encoder.mlp_units': 512,

  'decoder.mlp_keys': 'vector',
#   'decoder.mlp_units': 512,
    
  'encoder.cnn_keys': '$^',
  'decoder.cnn_keys': '$^',
    
#   'reward_head.units': 512,
#   'cont_head.units': 512,
    
  'model_opt.lr': 1e-4,
    
  'jax.platform': 'cpu',
  'wrapper.length': 0,
    
  'envs.amount': 8
})
# config = embodied.Flags(config).parse()

logdir = embodied.Path(config.logdir)
step = embodied.Counter()
logger = embodied.Logger(step, [
  embodied.logger.TerminalOutput(),
  embodied.logger.JSONLOutput(logdir, 'metrics.jsonl'),
  embodied.logger.TensorBoardOutput(logdir),
  # embodied.logger.WandBOutput(logdir.name, config),
  # embodied.logger.MLFlowOutput(logdir.name),
])

2024-06-05 17:25:17.899646: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Reading config


In [3]:
import gym
from embodied.envs import from_gym

from cyl.simulation_base.env import resume_env
import numpy as np

print("Setting Up Environment", flush =True)
sim_log_name = config.logdir.split("/")[-1]
env  = resume_env(plot=False, dump_CL=False, dump_vtu=False, dump_debug=10, sim_log_name = sim_log_name)


--------------------------------------------------------------------------
No OpenFabrics connection schemes reported that they were able to be
used on a specific port.  As such, the openib BTL (OpenFabrics
support) will be disabled for this port.

  Local host:           cx3-15-29
  Local device:         mlx5_0
  Local port:           1
  CPCs attempted:       rdmacm, udcm
--------------------------------------------------------------------------
bash: which: line 1: syntax error: unexpected end of file
bash: error importing function definition for `which'
bash: module: line 1: syntax error: unexpected end of file
bash: error importing function definition for `module'
bash: scl: line 1: syntax error: unexpected end of file
bash: error importing function definition for `scl'
bash: ml: line 1: syntax error: unexpected end of file
bash: error importing function definition for `ml'
bash: which: line 1: syntax error: unexpected end of file
bash: error importing function definition for `whi

iUFL can be obtained from https://github.com/MiroK/ufl-interpreter
Setting Up Environment
Number of probes: 64
test cwd env:  /rds/general/user/pg221/home/PhD_projects2/dreamer_flows/dreamer
Simulated 218 iterations before starting the control
--- done buffers initialisation ---


In [4]:
env = from_gym.FromGym(env, obs_key='vector')  # Or obs_key='vector'.
env = dreamerv3.wrap_env(env, config)
env = embodied.BatchEnv([env], parallel=False)

In [5]:
agent = dreamerv3.Agent(env.obs_space, env.act_space, step, config)

Encoder CNN shapes: {}
Encoder MLP shapes: {'vector': (66,)}
Decoder CNN shapes: {}
Decoder MLP shapes: {'vector': (66,)}
JAX devices (1): [CpuDevice(id=0)]
Policy devices: TFRT_CPU_0
Train devices:  TFRT_CPU_0


2024-06-05 17:25:37.835292: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Tracing train function.
Optimizer model_opt has 16,553,794 variables.
Optimizer actor_opt has 1,052,676 variables.
Optimizer critic_opt has 1,181,439 variables.


In [6]:
replay = embodied.replay.Uniform(
  config.batch_length, config.replay_size, logdir / 'replay')

args = embodied.Config(
  **config.run, logdir=config.logdir,
  batch_steps=config.batch_size * config.batch_length)

In [7]:
len(replay)

1193

Train the Model

In [ ]:
embodied.run.train(agent, env, replay, logger, args)

Logdir /rds/general/user/pg221/home/PhD_projects2/dreamer_flows/dreamer/logdir/notebook_test_cyl_ks_9
Observation space:
  vector           Space(dtype=float32, shape=(66,), low=-inf, high=inf)
  reward           Space(dtype=float32, shape=(), low=-inf, high=inf)
  is_first         Space(dtype=bool, shape=(), low=False, high=True)
  is_last          Space(dtype=bool, shape=(), low=False, high=True)
  is_terminal      Space(dtype=bool, shape=(), low=False, high=True)
Action space:
  action           Space(dtype=float32, shape=(2,), low=-1.0, high=1.0)
  reset            Space(dtype=bool, shape=(), low=False, high=True)
Prefill train dataset.
len replay in driver:  1193
max() in driver:  128
Done Prefill train dataset.
Found existing checkpoint.
Loading checkpoint: /rds/general/user/pg221/home/PhD_projects2/dreamer_flows/dreamer/logdir/notebook_test_cyl_ks_9/checkpoint.ckpt
Loaded checkpoint from 12331 seconds ago.
Start training loop.
step in driver:  0
Number of probes: 64
test cwd env

──────────────────────────────────────────────────── Step 601 ─────────────────────────────────────────────────────

train/action_mag 4.12 / train/action_max 4.12 / train/action_mean 0.02 / train/action_min -3.92 / train/action_std 
1.32 / train/actor_opt_actor_opt_grad_overflow 0 / train/actor_opt_actor_opt_grad_scale 1e4 / 
train/actor_opt_grad_norm 2.5e-4 / train/actor_opt_grad_steps 101 / train/actor_opt_loss 1220.43 / train/adv_mag 
0.21 / train/adv_max -0.02 / train/adv_mean -0.12 / train/adv_min -0.21 / train/adv_std 0.06 / train/cont_avg 1 / 
train/cont_loss_mean 3.7e-3 / train/cont_loss_std 1.7e-3 / train/cont_neg_acc nan / train/cont_neg_loss nan / 
train/cont_pos_acc 1 / train/cont_pos_loss 3.7e-3 / train/cont_pred 1 / train/cont_rate 1 / train/dyn_loss_mean 1 /
train/dyn_loss_std 0 / train/extr_critic_critic_opt_critic_opt_grad_overflow 0 / 
train/extr_critic_critic_opt_critic_opt_grad_scale 1e4 / train/extr_critic_critic_opt_grad_norm 18.33 / 
train/extr_critic_critic_opt_grad_steps 101 / train/extr_critic_critic_opt_loss 7.8e4 / train/extr_critic_mag 
3.6e-3 / train/extr_critic_max -3.1e-3 / train/extr_critic_mean -3.5e-3 / train/extr_critic_min -3.6e-3 / 
train/extr_critic_std 4.5e-5 / train/extr_return_normed_mag 0.13 / train/extr_return_normed_max 0.06 / 
train/extr_return_normed_mean -0.05 / train/extr_return_normed_min -0.13 / train/extr_return_normed_std 0.06 / 
train/extr_return_rate 0 / train/extr_return_raw_mag 0.21 / train/extr_return_raw_max -0.02 / 
train/extr_return_raw_mean -0.13 / train/extr_return_raw_min -0.21 / train/extr_return_raw_std 0.06 / 
train/extr_reward_mag 0.02 / train/extr_reward_max -0.02 / train/extr_reward_mean -0.02 / train/extr_reward_min 
-0.02 / train/extr_reward_std 2.2e-5 / train/model_loss_mean 1.8 / train/model_loss_std 0.17 / 
train/model_opt_grad_norm 8.59 / train/model_opt_grad_steps 101 / train/model_opt_loss 1.8e4 / 
train/model_opt_model_opt_grad_overflow 0 / train/model_opt_model_opt_grad_scale 1e4 / train/policy_entropy_mag 
2.83 / train/policy_entropy_max 2.83 / train/policy_entropy_mean 2.81 / train/policy_entropy_min 2.72 / 
train/policy_entropy_std 0.01 / train/policy_logprob_mag 11.39 / train/policy_logprob_max -1.76 / 
train/policy_logprob_mean -2.77 / train/policy_logprob_min -11.39 / train/policy_logprob_std 0.97 / 
train/policy_randomness_mag 1 / train/policy_randomness_max 1 / train/policy_randomness_mean 0.99 / 
train/policy_randomness_min 0.97 / train/policy_randomness_std 2.5e-3 / train/post_ent_mag 106.21 / 
train/post_ent_max 106.21 / train/post_ent_mean 106.08 / train/post_ent_min 105.95 / train/post_ent_std 0.05 / 
train/prior_ent_mag 106.65 / train/prior_ent_max 106.65 / train/prior_ent_mean 106.39 / train/prior_ent_min 106.16 
/ train/prior_ent_std 0.11 / train/rep_loss_mean 1 / train/rep_loss_std 0 / train/reward_avg -0.02 / 
train/reward_loss_mean 1.03 / train/reward_loss_std 0.12 / train/reward_max_data 0.1 / train/reward_max_pred 0.02 /
train/reward_neg_acc 1 / train/reward_neg_loss 1.03 / train/reward_pos_acc nan / train/reward_pos_loss nan / 
train/reward_pred -0.02 / train/reward_rate 0 / train/vector_loss_mean 0.16 / train/vector_loss_std 0.1 / 
train/params_agent/wm/model_opt 1.7e7 / train/params_agent/task_behavior/critic/critic_opt 1.2e6 / 
train/params_agent/task_behavior/ac/actor_opt 1.1e6 / replay/size 2386 / replay/inserts 0 / replay/samples 112 / 
replay/insert_wait_avg nan / replay/insert_wait_frac nan / replay/sample_wait_avg 1.8e-6 / replay/sample_wait_frac 
1 / timer/duration 164.39 / timer/logger.write_count 1 / timer/logger.write_total 3.1e-6 / timer/logger.write_frac 
1.9e-8 / timer/logger.write_avg 3.1e-6 / timer/logger.write_min 3.1e-6 / timer/logger.write_max 3.1e-6 / 
timer/replay.add_count 1201 / timer/replay.add_total 0.02 / timer/replay.add_frac 1.4e-4 / timer/replay.add_avg 
1.9e-5 / timer/replay.add_min 5.5e-6 / timer/replay.add_max 1.2e-3 / timer/checkpoint.load_count 1 / 
timer/checkpoint.load_total 0.16 / timer/checkpoint.load_frac 9.6e-4 / timer/checkpoint.load_avg 0.16 / 
timer/checkpoint.load_min 0.16 / timer/checkpoint.load


Creating new TensorBoard event file writer.
step in driver:  1
step in driver:  2
step in driver:  3
step in driver:  4
step in driver:  5
step in driver:  6
step in driver:  7
step in driver:  8
step in driver:  9
step in driver:  10
step in driver:  11
step in driver:  12
step in driver:  13
step in driver:  14
step in driver:  15
step in driver:  16
step in driver:  17
step in driver:  18
step in driver:  19
step in driver:  20
step in driver:  21
step in driver:  22
step in driver:  23
step in driver:  24
step in driver:  25
step in driver:  26
step in driver:  27
step in driver:  28
step in driver:  29


──────────────────────────────────────────────────── Step 630 ─────────────────────────────────────────────────────

...

episode/length 200 / episode/score -4.19 / episode/sum_abs_reward 4.2 / episode/reward_rate 0 / train/action_mag 
4.25 / train/action_max 4.25 / train/action_mean 0.93 / train/action_min -3.36 / train/action_std 1 / 
train/actor_opt_actor_opt_grad_overflow 0 / train/actor_opt_actor_opt_grad_scale 1e4 / train/actor_opt_grad_norm 
1.2e-4 / train/actor_opt_grad_steps 450 / train/actor_opt_loss 1395.88 / train/adv_mag 0.24 / train/adv_max -0.02 /
train/adv_mean -0.14 / train/adv_min -0.24 / train/adv_std 0.07 / train/cont_avg 0.99 / train/cont_loss_mean 0.05 /
train/cont_loss_std 0.5 / train/cont_neg_acc 0 / train/cont_neg_loss 5.71 / train/cont_pos_acc 1 / 
train/cont_pos_loss 3.3e-3 / train/cont_pred 1 / train/cont_rate 0.99 / train/dyn_loss_mean 1 / train/dyn_loss_std 
0 / train/extr_critic_critic_opt_critic_opt_grad_overflow 0 / train/extr_critic_critic_opt_critic_opt_grad_scale 
1e4 / train/extr_critic_critic_opt_grad_norm 6.01 / train/extr_critic_critic_opt_grad_steps 450 / 
train/extr_critic_critic_opt_loss 2.2e4 / train/extr_critic_mag 0.18 / train/extr_critic_max -0.18 / 
train/extr_critic_mean -0.18 / train/extr_critic_min -0.18 / train/extr_critic_std 1.9e-4 / 
train/extr_return_normed_mag 0.16 / train/extr_return_normed_max 0.16 / train/extr_return_normed_mean 0.04 / 
train/extr_return_normed_min -0.06 / train/extr_return_normed_std 0.07 / train/extr_return_rate 0 / 
train/extr_return_raw_mag 0.42 / train/extr_return_raw_max -0.21 / train/extr_return_raw_mean -0.33 / 
train/extr_return_raw_min -0.42 / train/extr_return_raw_std 0.07 / train/extr_reward_mag 0.02 / 
train/extr_reward_max -0.02 / train/extr_reward_mean -0.02 / train/extr_reward_min -0.02 / train/extr_reward_std 
2e-5 / train/model_loss_mean 1.21 / train/model_loss_std 0.59 / train/model_opt_grad_norm 6.63 / 
train/model_opt_grad_steps 450 / train/model_opt_loss 1.2e4 / train/model_opt_model_opt_grad_overflow 0 / 
train/model_opt_model_opt_grad_scale 1e4 / train/policy_entropy_mag 2.83 / train/policy_entropy_max 2.83 / 
train/policy_entropy_mean 2.83 / train/policy_entropy_min 2.79 / train/policy_entropy_std 3.1e-3 / 
train/policy_logprob_mag 11.87 / train/policy_logprob_max -1.82 / train/policy_logprob_mean -2.82 / 
train/policy_logprob_min -11.87 / train/policy_logprob_std 1.03 / train/policy_randomness_mag 1 / 
train/policy_randomness_max 1 / train/policy_randomness_mean 1 / train/policy_randomness_min 0.99 / 
train/policy_randomness_std 6.7e-4 / train/post_ent_mag 102.31 / train/post_ent_max 102.31 / train/post_ent_mean 
102.18 / train/post_ent_min 102.06 / train/post_ent_std 0.05 / train/prior_ent_mag 105.52 / train/prior_ent_max 
105.52 / train/prior_ent_mean 105.25 / train/prior_ent_min 104.97 / train/prior_ent_std 0.09 / train/rep_loss_mean 
1 / train/rep_loss_std 0 / train/reward_avg -0.02 / train/reward_loss_mean 0.46 / train/reward_loss_std 0.29 / 
train/reward_max_data 0.14 / train/reward_max_pred 0.02 / train/reward_neg_acc 1 / train/reward_neg_loss 0.46 / 
train/reward_pos_acc nan / train/reward_pos_loss nan / train/reward_pred -0.02 / train/reward_rate 0 / 
train/vector_loss_mean 0.1 / train/vector_loss_std 0.08 / stats/mean_log_entropy 2.82 / replay/size 3789 / 
replay/inserts 30 / replay/samples 128 / replay/insert_wait_avg 7.1e-6 / replay/insert_wait_frac 1 / 
replay/sample_wait_avg 2.2e-6 / replay/sample_wait_frac 1 / timer/duration 193.91 / timer/logger.write_count 1 / 
timer/logger.write_total 0.02 / timer/logger.write_frac 1e-4 / timer/logger.write_avg 0.02 / timer/logger.write_min
0.02 / timer/logger.write_max 0.02 / timer/replay.add_count 30 / timer/replay.add_total 0.01 / 
timer/replay.add_frac 5.5e-5 / timer/replay.add_avg 3.5e-4 / timer/replay.add_min 1.6e-4 / timer/replay.add_max 
7.7e-4 / timer/checkpoint.load_count 0 / timer/checkpoint.load_total 0 / timer/checkpoint.load_frac 0 / 
timer/env.step_count 30 / timer/env.step_total 183.44 / timer/env.step_frac 0.95 / timer/env.step_avg 6.11 / 
timer/env.step_min 5 / timer/env.step_


step in driver:  10
step in driver:  11
step in driver:  12
step in driver:  13
step in driver:  14
step in driver:  15
step in driver:  16
step in driver:  17
step in driver:  18
step in driver:  19
step in driver:  20
step in driver:  21
step in driver:  22
step in driver:  23
step in driver:  24
step in driver:  25
step in driver:  26
step in driver:  27
step in driver:  28
step in driver:  29
step in driver:  30
step in driver:  31
step in driver:  32
step in driver:  33
step in driver:  34
step in driver:  35
step in driver:  36
step in driver:  37
step in driver:  38
step in driver:  39


──────────────────────────────────────────────────── Step 2040 ────────────────────────────────────────────────────

train/action_mag 4.53 / train/action_max 4.53 / train/action_mean 0.97 / train/action_min -2.57 / train/action_std 
0.99 / train/actor_opt_actor_opt_grad_overflow 0 / train/actor_opt_actor_opt_grad_scale 1e4 / 
train/actor_opt_grad_norm 1.1e-4 / train/actor_opt_grad_steps 460 / train/actor_opt_loss 1444.44 / train/adv_mag 
0.25 / train/adv_max -0.02 / train/adv_mean -0.15 / train/adv_min -0.25 / train/adv_std 0.07 / train/cont_avg 1 / 
train/cont_loss_mean 3.2e-3 / train/cont_loss_std 1.1e-3 / train/cont_neg_acc nan / train/cont_neg_loss nan / 
train/cont_pos_acc 1 / train/cont_pos_loss 3.2e-3 / train/cont_pred 1 / train/cont_rate 1 / train/dyn_loss_mean 1 /
train/dyn_loss_std 0 / train/extr_critic_critic_opt_critic_opt_grad_overflow 0 / 
train/extr_critic_critic_opt_critic_opt_grad_scale 1e4 / train/extr_critic_critic_opt_grad_norm 6.05 / 
train/extr_critic_critic_opt_grad_steps 460 / train/extr_critic_critic_opt_loss 2.2e4 / train/extr_critic_mag 0.19 
/ train/extr_critic_max -0.19 / train/extr_critic_mean -0.19 / train/extr_critic_min -0.19 / train/extr_critic_std 
1.8e-4 / train/extr_return_normed_mag 0.16 / train/extr_return_normed_max 0.16 / train/extr_return_normed_mean 0.03
/ train/extr_return_normed_min -0.06 / train/extr_return_normed_std 0.07 / train/extr_return_rate 0 / 
train/extr_return_raw_mag 0.43 / train/extr_return_raw_max -0.21 / train/extr_return_raw_mean -0.33 / 
train/extr_return_raw_min -0.43 / train/extr_return_raw_std 0.07 / train/extr_reward_mag 0.02 / 
train/extr_reward_max -0.02 / train/extr_reward_mean -0.02 / train/extr_reward_min -0.02 / train/extr_reward_std 
2.7e-5 / train/model_loss_mean 1.13 / train/model_loss_std 0.23 / train/model_opt_grad_norm 6.85 / 
train/model_opt_grad_steps 460 / train/model_opt_loss 1.1e4 / train/model_opt_model_opt_grad_overflow 0 / 
train/model_opt_model_opt_grad_scale 1e4 / train/policy_entropy_mag 2.83 / train/policy_entropy_max 2.83 / 
train/policy_entropy_mean 2.83 / train/policy_entropy_min 2.81 / train/policy_entropy_std 2.7e-3 / 
train/policy_logprob_mag 8.62 / train/policy_logprob_max -1.83 / train/policy_logprob_mean -2.82 / 
train/policy_logprob_min -8.62 / train/policy_logprob_std 1 / train/policy_randomness_mag 1 / 
train/policy_randomness_max 1 / train/policy_randomness_mean 1 / train/policy_randomness_min 0.99 / 
train/policy_randomness_std 5.8e-4 / train/post_ent_mag 102.34 / train/post_ent_max 102.34 / train/post_ent_mean 
102.09 / train/post_ent_min 101.95 / train/post_ent_std 0.06 / train/prior_ent_mag 105.35 / train/prior_ent_max 
105.35 / train/prior_ent_mean 105.2 / train/prior_ent_min 105.02 / train/prior_ent_std 0.08 / train/rep_loss_mean 1
/ train/rep_loss_std 0 / train/reward_avg -0.02 / train/reward_loss_mean 0.43 / train/reward_loss_std 0.21 / 
train/reward_max_data 0.12 / train/reward_max_pred 0.02 / train/reward_neg_acc 1 / train/reward_neg_loss 0.43 / 
train/reward_pos_acc nan / train/reward_pos_loss nan / train/reward_pred -0.02 / train/reward_rate 0 / 
train/vector_loss_mean 0.09 / train/vector_loss_std 0.08 / replay/size 3819 / replay/inserts 30 / replay/samples 
112 / replay/insert_wait_avg 5.5e-6 / replay/insert_wait_frac 1 / replay/sample_wait_avg 2.1e-6 / 
replay/sample_wait_frac 1 / timer/duration 157.74 / timer/logger.write_count 1 / timer/logger.write_total 0.02 / 
timer/logger.write_frac 1.4e-4 / timer/logger.write_avg 0.02 / timer/logger.write_min 0.02 / timer/logger.write_max
0.02 / timer/replay.add_count 30 / timer/replay.add_total 0.01 / timer/replay.add_frac 6.4e-5 / 
timer/replay.add_avg 3.3e-4 / timer/replay.add_min 1.6e-4 / timer/replay.add_max 4.7e-4 / 
timer/checkpoint.load_count 0 / timer/checkpoint.load_total 0 / timer/checkpoint.load_frac 0 / timer/env.step_count
30 / timer/env.step_total 148.71 / timer/env.step_frac 0.94 / timer/env.step_avg 4.96 / timer/env.step_min 4.85 / 
timer/env.step_max 5.32 / timer/agent.policy_count 30 / timer/agent.policy_total 0.32 / timer/agent.policy_frac 
2e-3 / timer/agent.policy_


step in driver:  40
step in driver:  41
step in driver:  42
step in driver:  43
step in driver:  44
step in driver:  45
step in driver:  46
step in driver:  47
step in driver:  48
step in driver:  49
step in driver:  50
step in driver:  51
step in driver:  52
step in driver:  53
step in driver:  54
step in driver:  55
step in driver:  56
